### Importación de librerías

In [43]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim


### En caso de estar disponible se usa la gpu, si no se usa la cpu.

In [44]:
# Comprueba si la GPU está disponible y utiliza CUDA si es posible
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda:0


### Transformación y carga de datos CIFAR-10

Transformación:
- transform.Compose() combina varias transformaciones en una sola función para poder aplicarlas de forma secuencial.

- ToTensor() combierte las imágenes a tensores y cambia e rango de valores de 0-255 a 0-1.

- Normalize() normaliza los canales RGB de las imagenes (en este caso estableca las medias y desviaciones típicas a 0.5)

Carga de datos:
- torchvision.datasets.CIFAR10() carga el conjunto de datos.

- .DataLoader() configura el batch size a 64 y mezcla los datos del train set. Usa 2 workers (2 subprocesos) para mejorar la velocidad de carga.

In [45]:
# Transformaciones para el conjunto de datos CIFAR-10
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Cargar los conjuntos de datos de entrenamiento y prueba
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


### Cargar un Modelo Preentrenado y Modificarlo para CIFAR-10

#### Cargar el Modelo Preentrenado:
- models.resnet18(): Carga una versión preentrenada de ResNet-18.

#### Modificar la Capa de Salida:
- num_ftrs = model.fc.in_features: Obtiene el número de características de entrada de la última capa lineal del modelo ResNet-18, necesitamos modificarla para que coincida con el número de clases en CIFAR-10.
- model.fc = nn.Linear(num_ftrs, 10): reemplaza la ultima capa con una nueva que tiene 10 salidas.

#### Mover el modelo a la GPU
- model.to(device): Mueve el modelo a la GPU.



In [46]:
# Cargar un modelo preentrenado y modificarlo
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10) # 10 clases en CIFAR-10

# Mover el modelo a la GPU
model.to(device)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### Definición de Criterio de Pérdida y Optimizador

#### Criterio de Pérdida:
- criterion = nn.CrossEntropyLoss(): Define el criterio de pérdida como la entropía cruzada, utilizada para problemas de clasificación.

#### Optimizador:
- optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9): Configura el optimizador como el Descenso de Gradiente Estocástico (SGD). `lr` es la tasa de aprendizaje, y `momentum` es un término que ayuda a acelerar el SGD en la dirección correcta y amortiguar las oscilaciones. El alto momentum de 0.9 contribuye a que el optimizador mantenga la dirección en la que la pérdida decrece más rápidamente.


In [47]:
# Definir criterio de pérdida y optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


### Bucle de entrenamiento

-Durante cada época, el modelo se pone en modo de entrenamiento y procesa los datos en batches. Para cada batch, realiza una pasada hacia adelante generando predicciones, calcula la pérdida con respecto a las verdaderas etiquetas, realiza una pasada hacia atrás para calcular los gradientes, y luego ajusta los pesos con un paso de optimización

In [48]:
num_epochs = 15 # Define el número de épocas

for epoch in range(num_epochs):
    model.train()  # Poner el modelo en modo de entrenamiento
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device) # Mover los datos a la GPU

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Época {epoch + 1}, Pérdida: {running_loss / len(trainloader):.3f}')



print('Entrenamiento finalizado')


Época 1, Pérdida: 1.034
Época 2, Pérdida: 0.631
Época 3, Pérdida: 0.477
Época 4, Pérdida: 0.372
Época 5, Pérdida: 0.287
Época 6, Pérdida: 0.228
Época 7, Pérdida: 0.178
Época 8, Pérdida: 0.137
Época 9, Pérdida: 0.113
Época 10, Pérdida: 0.097
Época 11, Pérdida: 0.086
Época 12, Pérdida: 0.079
Época 13, Pérdida: 0.068
Época 14, Pérdida: 0.058
Época 15, Pérdida: 0.060
Entrenamiento finalizado


### Se comprueba la precisión del modelo con el conjunto de test


In [49]:
# Evaluar el modelo
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Precisión del modelo en el conjunto de datos de prueba CIFAR-10: {100 * correct // total}%')

Precisión del modelo en el conjunto de datos de prueba CIFAR-10: 79%


#Mejorar Modelo

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import random_split

# Configuración de dispositivo (GPU si está disponible)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


### Igual que antes pero aplicando algo de data augmentation con RandomHorizontalflip y Rndom Rotation (10 es el máximo de grados que puede rotar una imagen)

In [3]:
# Transformaciones para el conjunto de datos CIFAR-10
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Cargar los conjuntos de datos de entrenamiento y prueba
trainset = torchvision.datasets.CIFAR10(root='./data_2', train=True,
                                        download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data_2', train=False,
                                       download=True, transform=transform)

# Dividir el conjunto de entrenamiento en subconjuntos de entrenamiento y validación
train_size = int(0.8 * len(trainset))
val_size = len(trainset) - train_size
train_dataset, val_dataset = random_split(trainset, [train_size, val_size])

trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
valloader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)


100%|██████████| 170498071/170498071 [00:01<00:00, 106008470.82it/s]


Extracting ./data_2/cifar-10-python.tar.gz to ./data_2
Files already downloaded and verified


### En vez de usar resnet 18 esamos el 34 y añadimos dropout


In [4]:
# Definición del modelo
model = models.resnet34(pretrained=True)
num_ftrs = model.fc.in_features

# Añadir dropout a la capa final
model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(num_ftrs, 10)  # 10 clases para CIFAR-10
)

model = model.to(device)

# Criterio de pérdida y optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:01<00:00, 70.7MB/s]


### Se añaden mas épocas, he puesto 100(claramente no hacen falta tantas, de hecho llega a 83% en la época 45).

In [5]:
num_epochs = 100

for epoch in range(num_epochs):
    # Entrenamiento
    model.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Validación
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for data in valloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Época {epoch + 1}, Pérdida de Entrenamiento: {running_loss / len(trainloader)}, Pérdida de Validación: {val_loss / len(valloader)}, Precisión de Validación: {100 * correct / total}%')




Época 1, Pérdida de Entrenamiento: 1.2816181645393372, Pérdida de Validación: 0.865786459795229, Precisión de Validación: 69.6%
Época 2, Pérdida de Entrenamiento: 0.8284113910198212, Pérdida de Validación: 0.7187041763667088, Precisión de Validación: 75.24%
Época 3, Pérdida de Entrenamiento: 0.6968674880504608, Pérdida de Validación: 0.6596883548672792, Precisión de Validación: 76.96%
Época 4, Pérdida de Entrenamiento: 0.6156878041744233, Pérdida de Validación: 0.6083787544897408, Precisión de Validación: 78.62%
Época 5, Pérdida de Entrenamiento: 0.5580404947042465, Pérdida de Validación: 0.6120230948469442, Precisión de Validación: 78.67%
Época 6, Pérdida de Entrenamiento: 0.5112025663375854, Pérdida de Validación: 0.5788823654697199, Precisión de Validación: 80.17%
Época 7, Pérdida de Entrenamiento: 0.47090019094944, Pérdida de Validación: 0.580899579509808, Precisión de Validación: 80.17%
Época 8, Pérdida de Entrenamiento: 0.43528888432979584, Pérdida de Validación: 0.56749248115500

In [6]:

# Evaluar el modelo en el conjunto de prueba
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Precisión del modelo en el conjunto de datos de prueba CIFAR-10: {100 * correct // total}%')


Precisión del modelo en el conjunto de datos de prueba CIFAR-10: 83%
